In [1]:
import numpy,datetime,re
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher

In [2]:
tentative_engagements = []
free_imdbs = []
corresponacy1 = []

In [3]:
def importdatasets():
    #READ DATA:
    IMDB_DF = pd.read_csv("imdb.csv")
    #for task2
    IMDB_DF_SAMPLE = IMDB_DF.sample(1000).astype(str)

    imdb_header = list(IMDB_DF.columns.values)

    #imdb_header for task 1
    tbl1 = np.array(imdb_header)

    ROTTEN_TOMATOES_DF = pd.read_csv("rotten_tomatoes.csv")
    #for task2
    ROTTEN_TOMATOES_DF_SAMPLE = ROTTEN_TOMATOES_DF.sample(1000).astype(str)
    rt_header = list(ROTTEN_TOMATOES_DF.columns.values)
    #rt_header for task1
    tbl2 = np.array(rt_header)

    return IMDB_DF_SAMPLE,tbl1,ROTTEN_TOMATOES_DF_SAMPLE,tbl2

In [4]:
def matchingtablebyheaders(tbl1,tbl2):
          #creation array to add sort index of matching
    # For matching
    Pref=np.zeros((len(tbl1),len(tbl2)))
        #assign the matching ratio between tbl1 and tbl2
    for i in range(len(tbl1)):
        for j in range(len(tbl2)):
            Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i],tbl2[j]).ratio())
    #df = pd.DataFrame(Pref)
    #df.to_csv("file_path.csv")
    return Pref

In [5]:
def listofpreferencespertable(tbl1,tbl2,Pref):
        #for Tomatoes
    ipref_tom=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
    #for IMdb
    ipref_imdb=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
    pref_imdb=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
    preferred_rankings_tomatoes = {}
    preferred_rankings_imdb = {}
    #create list of preferred_rankings_tomatoes with header
    #assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl2 (17)
    for i in range(len(tbl2)):
        ipref_tom[:,i]=np.argsort(Pref[:,i])
        ipref_tom[:,i]=ipref_tom[::-1,i]
        preferred_rankings_tomatoes[i] = ipref_tom[:,i]
        #pref_tom[:,i]=np.sort(Pref[:,i])
        #pref_tom[:,i]=pref_tom[::-1,i]
    #ipref_tom

    #create list of preferred_rankings_Imdb with header
    #assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl1 (13)
    for i in range(len(tbl1)):
        #get indexs values of rows and store them in the rows as well
        ipref_imdb[:,i] = np.argsort(Pref[i,:])
        ipref_imdb[:,i] = ipref_imdb[::-1,i]
        preferred_rankings_imdb[i] = ipref_imdb[:,i]
        #pref_imdb[:,i] = np.sort(Pref[i,:])
      #  pref_imdb[:,i] = pref_imdb[::-1,i]
    #ipref_imdb
    return preferred_rankings_imdb,preferred_rankings_tomatoes

In [6]:
#Imdb which still need to propose and get accepted successfully
#Keep track of the headers that "may" end up together
def init_free_imdbs(preferred_rankings_imdb):
    #corresponacy1 = list.clear()
    #'''Initialize the arrays of tomates and imdbs to represent that they're all initially free and not engaged'''
    for imdb in preferred_rankings_imdb.keys():
        free_imdbs.append(imdb)

In [7]:
def begin_matching(imdb,preferred_rankings_imdb,preferred_rankings_tomatoes):
    #  '''Find the first free Tomaten available to a Imdb at any given time'''
    #print("DEALING WITH %s"%(imdb))
    #print(preferred_rankings_imdb[imdb])
    for tomate in preferred_rankings_imdb[imdb]:
        #Boolean for whether tomate is taken or not
        taken_match = [couple for couple in tentative_engagements if tomate == couple [:][1]]
        if (len(taken_match) == 0):
            #tentatively engage the man and woman
            tentative_engagements.append([imdb, tomate])
            free_imdbs.remove(imdb)
            #print('%s is no longer a free imdb and is now tentatively engaged to %s'%( imdb, tomate))
            break
        elif (len(taken_match) > 0):
            #print('%s is taken already..'%(tomate))
            #print('%s tttt..'%(taken_match[0][0]))
            current_imdb = list(preferred_rankings_tomatoes[tomate].tolist()).index(taken_match[0][0])
            potential_imdb = list(preferred_rankings_tomatoes[tomate].tolist()).index(imdb)
            #print('potential_imdb %s..'%(current_imdb))
            if (current_imdb < potential_imdb):
                ii=[]
                #print('Tomataen\'s satisfied with %s..'%(taken_match[0][0]))
            else: 
                #print('%s is better than %s'%(imdb, taken_match[0][0]))
                #print('Making %s free again.. and tentatively engaging %s and %s'%(taken_match[0][0], imdb, tomate))
                #The new imdb is engaged
                free_imdbs.remove(imdb)
                #The old imdb is now single
                free_imdbs.append(taken_match[0][0])
                #Update the fiance of the tomaten (tentatively)
                taken_match[0][0] = imdb
                break

In [8]:
def stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes):
    '''Matching algorithm until stable match terminates'''
    while (len(free_imdbs) > 0):
        for imdb in free_imdbs:
            begin_matching(imdb,preferred_rankings_imdb,preferred_rankings_tomatoes)

In [9]:
def check_short_matching(COL1,COL2):
    rate = 0.0
    if str(COL1) == 'nan' or str(COL2) == 'nan' :
        return 0
    COL1 = str(COL1)
    COL2 = str(COL2)
    sw = (fuzz.SequenceMatcher(None,COL1, COL2).ratio())
    return sw #/(max(len(COL1),len(COL2))*1.0)

In [10]:
def check_match(st1,st2):
    str1 = np.array((st1.values).astype(str))
    str2 = np.array((st2.values).astype(str))
    ret = np.zeros((len(str1),len(str2)))
    for i in range(len(str1)):
        for j in range(len(str2)):
            ret[i,j] = check_short_matching(str1[i],str2[j])
    return numpy.average(ret)

In [11]:
def matchingtablebyvalues(IMDB_DF_SAMPLE,ROTTEN_TOMATOES_DF_SAMPLE,tbl1,tbl2):
    dd = []
    ret = []
    value1 = []
    value2 = []
    index_i = 0
    Pref=np.zeros((len(tbl1),len(tbl2)))
    for i in IMDB_DF_SAMPLE:
        dd = []
        index_j = 0
        for j in ROTTEN_TOMATOES_DF_SAMPLE:
            table1 = IMDB_DF_SAMPLE[i]
            table2 = ROTTEN_TOMATOES_DF_SAMPLE[j]
            value1 = (table1.values).astype(str)
            value2 = (table2.values).astype(str)
            #dd.append([ 100* (check_match(table1, table2)),i,j])
            Pref[index_i,index_j] = 100 * (np.float(check_match(table1, table2)))
            index_j+=1
        index_i+=1
    return Pref

In [12]:
def outputforcolumns(tbl1,tbl2,Pref):
    i = 0
    for item in tentative_engagements:
        matching=Pref[item[0],item[1]]
        if(matching > 50.0):
            corresponacy1.append(item)
            print(i ,'. CORRECT \t(',tbl1[item[0]],tbl2[item[1]],')with % of matching',matching)
            i+=1
        else:
                print(i,'. Wrong \t',tbl1[item[0]],tbl2[item[1]],'with % of matching',matching)
                i+=1
    found =len(corresponacy1) - 1
    print('A:Number of the discovered correspondences that are in G(withoutID)',found)
    print('B:Number of all the discovered correspondences',len(corresponacy1))
    print('precision(A/B):',found/len(corresponacy1)*100)
    print('recall(A/Number of all the actual correspondences = 10):',(found/10)*100)
    

In [13]:
def main1():
    tentative_engagements.clear()
    free_imdbs.clear()
    corresponacy1.clear()
    #add data
    IMDB_DF_SAMPLE,tbl1,ROTTEN_TOMATOES_DF_SAMPLE,tbl2=importdatasets()
    #creation templates
    Pref=matchingtablebyheaders(tbl1,tbl2)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
   # print(preferred_rankings_tomatoes)
    init_free_imdbs(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    print(tentative_engagements)
    outputforcolumns(tbl1,tbl2,Pref)
    
    
main1()

[[0, 0], [2, 2], [4, 4], [6, 7], [8, 11], [10, 14], [12, 16], [1, 1], [5, 5], [9, 12], [3, 3], [11, 6], [7, 10]]
0 . CORRECT 	( Id Id )with % of matching 100.0
1 . CORRECT 	( YearRange Year )with % of matching 61.0
2 . CORRECT 	( Director Director )with % of matching 100.0
3 . CORRECT 	( Cast Cast )with % of matching 100.0
4 . CORRECT 	( RatingValue RatingValue )with % of matching 100.0
5 . CORRECT 	( Genre Genre )with % of matching 100.0
6 . CORRECT 	( Description Description )with % of matching 100.0
7 . CORRECT 	( Name Name )with % of matching 100.0
8 . CORRECT 	( Creator Creator )with % of matching 100.0
9 . Wrong 	 ContentRating RatingCount with % of matching 50.0
10 . CORRECT 	( ReleaseDate Release Date )with % of matching 95.0
11 . Wrong 	 Url Actors with % of matching 22.0
12 . CORRECT 	( Duration Duration )with % of matching 100.0
A:Number of the discovered correspondences that are in G(withoutID) 10
B:Number of all the discovered correspondences 11
precision(A/B): 90.90909090

In [14]:
def outputforvalues(tbl1,tbl2,Pref):
    i = 0
    for item in tentative_engagements:
        matching=Pref[item[0],item[1]]
        if(matching > 0.0):
            corresponacy1.append(item)
            print(i ,'. CORRECT \t(',tbl1[item[0]],tbl2[item[1]],')with % of matching',matching)
            i+=1
        else:
                print(i,'. Wrong \t',tbl1[item[0]],tbl2[item[1]],'with % of matching',matching)
                i+=1
    found =len(corresponacy1) - 1
    print('A:Number of the discovered correspondences that are in G(withoutID)',found)
    print('B:Number of all the discovered correspondences',len(corresponacy1))
    print('precision(A/B):',found/len(corresponacy1)*100)
    print('recall(A/Number of all the actual correspondences = 10):',(found/10)*100)

In [15]:
def main2():
    entative_engagements.clear()
    free_imdbs.clear()
    corresponacy1.clear()
    #add data
    IMDB_DF_SAMPLE,tbl1,ROTTEN_TOMATOES_DF_SAMPLE,tbl2=importdatasets()
    #creation templates
    Pref=matchingtablebyvalues(IMDB_DF_SAMPLE,ROTTEN_TOMATOES_DF_SAMPLE,tbl1,tbl2)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    print(preferred_rankings_tomatoes)
    init_free_imdbs(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)

    
    
main2()

[[4, 4], [3, 3], [6, 7], [8, 11], [10, 14], [12, 16], [1, 1], [5, 6], [0, 5], [11, 15], [2, 0], [7, 10], [9, 13]]
0 . CORRECT 	( Director Director )with % of matching 26.295498299144633
1 . CORRECT 	( ReleaseDate Release Date )with % of matching 27.268117670702097
2 . CORRECT 	( Cast Cast )with % of matching 19.79445685124734
3 . CORRECT 	( RatingValue RatingValue )with % of matching 15.71717797619048
4 . CORRECT 	( Genre Genre )with % of matching 29.982142853807957
5 . CORRECT 	( Description Description )with % of matching 26.49840470178692
6 . CORRECT 	( Name Name )with % of matching 24.175764252512412
7 . CORRECT 	( Creator Actors )with % of matching 23.971470279727424
8 . CORRECT 	( Id Creator )with % of matching 19.408657598462185
9 . CORRECT 	( Url Filming Locations )with % of matching 16.38309800624926
10 . CORRECT 	( YearRange Id )with % of matching 25.43847826086956
11 . CORRECT 	( Duration Duration )with % of matching 23.639415254161033
12 . CORRECT 	( ContentRating ReviewCou

In [16]:
df = pd.DataFrame(tentative_engagements)
df.to_csv("tentative_engagements.csv")

In [22]:

df.to_csv("Prefbyvalue.csv")


AttributeError: 'function' object has no attribute 'Pref'